# Interval methods for sysmte of equations 

In [90]:
!pip install -q pyinterval

In [91]:
from interval import interval
from interval import imath
import pandas as pd
import numpy as np
import math

In [92]:
def calculate_determinant(m):
    return m[0][0]*m[1][1] - m[0][1]*m[1][0]

def calculate_width(interv):
    return sum([x.sup - x.inf for x in interv])

def calculate_midpoint(interv):
    return (interv[0].sup + interv[0].inf) / 2

def interval_to_str(interv):
    return f"[{interv[0].inf:9.7f}, {interv[0].sup:9.7f}]"

def half_interval(x): # subinterval
    mid = calculate_midpoint(x)
    return (interval[x[0].inf, mid], interval[mid, x[0].sup])

def to_table(x, x_width, y, y_width, root, tbl):
    tbl.loc[len(tbl)] = [interval_to_str(x), x_width, interval_to_str(y), y_width, root]

## Moore's method modification 

In [93]:
def moore_system(f, df, intervals, e, tbl):
    x, y = intervals
    x_width = calculate_width(x)
    y_width = calculate_width(y)

    # If no 0 in F(X, Y) -> no root
    f_xy = f(x, y)
    if 0 not in f_xy[0] or 0 not in f_xy[1]:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    # Return result if tolerance is satisfied
    if max(x_width, y_width) < e:
        to_table(x, x_width, y, y_width, 'Root', tbl)
        return

    # Half if 0 in F'(X, Y)
    x_middle = calculate_midpoint(x)
    y_middle = calculate_midpoint(y)
    df_xy = df(x, y)
    df_xy_det = interval(calculate_determinant(df_xy))

    if 0 in df_xy_det:
        x_left, x_right = half_interval(x)
        y_left, y_right = half_interval(y)
        if x_width < e: # half only Y
            to_table(x, x_width, y, y_width, '', tbl)
            moore_system(f, df, (x, y_left), e, tbl)
            moore_system(f, df, (x, y_right), e, tbl)
        elif y_width < e: # half only X
            to_table(x, x_width, y, y_width, '', tbl)
            moore_system(f, df, (x_left, y), e, tbl)
            moore_system(f, df, (x_right, y), e, tbl)
        else: # half everything
            to_table(x, x_width, y, y_width, '', tbl)
            moore_system(f, df, (x_left, y_left), e, tbl)
            moore_system(f, df, (x_left, y_right), e, tbl)
            moore_system(f, df, (x_right, y_left), e, tbl)
            moore_system(f, df, (x_right, y_right), e, tbl)
        return

    f1m, f2m = f(x_middle, y_middle)
    df1x, df1y, df2x, df2y = *df_xy[0], *df_xy[1]
    U_x = x_middle + ((-df2y/df_xy_det)*f1m + (df1y/df_xy_det)*f2m)
    U_y = y_middle + ((df2x/df_xy_det)*f1m + (-df1x/df_xy_det)*f2m)
    x_next = U_x & x
    y_next = U_y & y

    # Stop if x_i+1 or y_i+1 is empty
    if not x_next or not y_next:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    # [Step 4]: Continue with narrowed interval
    to_table(x, x_width, y, y_width, '', tbl)
    moore_system(f, df, (x_next, y_next), e, tbl)

In [94]:
func = lambda x, y: (x**2 - y - 3, x - y - 1)
dfunc = lambda x, y: ((2*x, -1), (1, -1))
initial_intervals = (interval[-4, 3.5], interval[-3.7, 5.4])
e = 1e-6

In [95]:
moore_system_result = pd.DataFrame(columns=['x', 'x width', 'y', 'y width', 'root'])
moore_system(func, dfunc, initial_intervals, e, moore_system_result)
moore_system_result

,x,x width,y,y width,root
0,"[-4.0000000, 3.5000000]",7.500000e+00,"[-3.7000000, 5.4000000]",9.100000e+00,
1,"[-4.0000000, -0.2500000]",3.750000e+00,"[-3.7000000, 0.8500000]",4.550000e+00,
2,"[-1.6093750, -0.2500000]",1.359375e+00,"[-3.7000000, 0.4409722]",4.140972e+00,
3,"[-1.1959801, -0.8495521]",3.464279e-01,"[-2.6110813, -1.7850704]",8.260109e-01,
4,"[-1.0157451, -0.9840028]",3.174229e-02,"[-2.0382874, -1.9522683]",8.601917e-02,
5,"[-1.0000338, -0.9999663]",6.752615e-05,"[-2.0000997, -1.9999024]",1.972274e-04,
6,"[-1.0000000, -1.0000000]",3.176726e-11,"[-2.0000000, -2.0000000]",9.355450e-11,Root
7,"[-4.0000000, -0.2500000]",3.750000e+00,"[0.8500000, 5.4000000]",4.550000e+00,
8,"[-0.2500000, 3.5000000]",3.750000e+00,"[-3.7000000, 0.8500000]",4.550000e+00,
9,"[-0.2500000, 1.6250000]",1.875000e+00,"[-3.7000000, -1.4250000]",2.275000e+00,


## Hansen's method modification 

In [96]:
def hansen_system(f, df, intervals, e, tbl):
    x, y = intervals
    x_width = calculate_width(x)
    y_width = calculate_width(y)    

    # If no 0 in F(x, y) -> no root
    f_xy = f(x, y)
    if 0 not in f_xy[0] or 0 not in f_xy[1]:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    # If interval width less than epsilon -> root
    if max(x_width, y_width) < e:
        to_table(x, x_width, y, y_width, 'Root', tbl)
        return

    x_middle = calculate_midpoint(x)
    y_middle = calculate_midpoint(y)
    df_xy = df(x, y)
    df_xy_det = interval(calculate_determinant(df_xy))  
    f1m, f2m = f(x_middle, y_middle)

    # Check midpoints
    if f1m == 0.0 and f2m == 0.0:
        to_table(x, x_width, y, y_width, '', tbl)
        x_left, x_right = half_interval(x)
        y_left, y_right = half_interval(y)
        hansen_system(f, df, (x_left, y_left), e, tbl)
        hansen_system(f, df, (x_left, y_right), e, tbl)
        hansen_system(f, df, (x_right, y_left), e, tbl)
        hansen_system(f, df, (x_right, y_right), e, tbl)
        return

    df1x, df1y, df2x, df2y = *df_xy[0], *df_xy[1]
    U_x = x_middle + ((-df2y/df_xy_det)*f1m + (df1y/df_xy_det)*f2m)
    U_y = y_middle + ((df2x/df_xy_det)*f1m + (-df1x/df_xy_det)*f2m)
    x_next = U_x & x
    y_next = U_y & y

    # Stop if x_i+1 or y_i+1 is empty
    if not x_next or not y_next:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    # Force split if X and Y haven't changed
    if x == x_next and y == y_next:
        if(x_width > y_width):
            to_table(x, x_width, y, y_width, '', tbl)
            x_left, x_right = half_interval(x)
            hansen_system(f, df, (x_left, y), e, tbl)
            hansen_system(f, df, (x_right, y), e, tbl)
        else:
            to_table(x, x_width, y, y_width, '', tbl)
            y_left, y_right = half_interval(y)
            hansen_system(f, df, (x, y_left), e, tbl)
            hansen_system(f, df, (x, y_right), e, tbl)
        return

    # Continue with narrowed intervals
    to_table(x, x_width, y, y_width, '', tbl)
    for xi in x_next:
        for yi in y_next:
            hansen_system(f, df, (interval(xi), interval(yi)), e, tbl)

In [97]:
func = lambda x, y: (x**2 - y - 3, x - y - 1)
dfunc = lambda x, y: ((2*x, -1), (1, -1))
init_intervals = (interval[-1.3, 2.1], interval[-2.2, 2.4])
e = 1e-6

In [98]:
hansen_system_result = pd.DataFrame(columns=['x', 'x width', 'y', 'y width', 'root'])
hansen_system(func, dfunc, init_intervals, e, hansen_system_result)
hansen_system_result

,x,x width,y,y width,root
0,"[-1.3000000, 2.1000000]",3.400000e+00,"[-2.2000000, 2.4000000]",4.600000e+00,
1,"[-1.3000000, 2.1000000]",3.400000e+00,"[-2.2000000, 0.1000000]",2.300000e+00,
2,"[-1.3000000, 0.4000000]",1.700000e+00,"[-2.2000000, 0.1000000]",2.300000e+00,
3,"[-1.3000000, 0.4000000]",1.700000e+00,"[-2.2000000, 0.0645833]",2.264583e+00,
4,"[-1.3000000, 0.4000000]",1.700000e+00,"[-2.2000000, -0.0190394]",2.180961e+00,
5,"[-1.3000000, 0.4000000]",1.700000e+00,"[-2.2000000, -0.2164818]",1.983518e+00,
6,"[-1.3000000, 0.3173346]",1.617335e+00,"[-2.2000000, -0.6826654]",1.517335e+00,
7,"[-1.3000000, -0.7203758]",5.796242e-01,"[-2.2000000, -1.7203758]",4.796242e-01,
8,"[-1.0042197, -0.9950725]",9.147178e-03,"[-2.0213709, -1.9855685]",3.580242e-02,
9,"[-1.0000067, -0.9999933]",1.339312e-05,"[-2.0000222, -1.9999777]",4.448570e-05,


## Krawczyk's method modification 

In [99]:
def krawczyk_system(f, df, intervals, e, tbl):
    x, y = intervals
    x_width = calculate_width(x)
    y_width = calculate_width(y)

    # If no 0 in F(X, Y) -> no root
    f_xy = f(x, y)
    if 0 not in f_xy[0] or 0 not in f_xy[1]:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    # If interval width less than epsilon -> root
    if max(x_width, y_width) < e:
        to_table(x, x_width, y, y_width, 'Root', tbl)
        return

    # Half if 0 in F'(X, Y)
    x_middle = calculate_midpoint(x)
    y_middle = calculate_midpoint(y)
    df_xy = df(x, y)
    df_xy_det = interval(calculate_determinant(df_xy))

    if 0 in df_xy_det:
        x_left, x_right = half_interval(x)
        y_left, y_rigth = half_interval(y)
        if x_width < e: # half only Y
            to_table(x, x_width, y, y_width, '', tbl)
            krawczyk_system(f, df, (x, y_left), e, tbl)
            krawczyk_system(f, df, (x, y_right), e, tbl)
        elif y_width < e: # half only X
            to_table(x, x_width, y, y_width, '', tbl)
            krawczyk_system(f, df, (x_left, y), e, tbl)
            krawczyk_system(f, df, (x_right, y), e, tbl)
        else: # half everything
            to_table(x, x_width, y, y_width, '', tbl)
            krawczyk_system(f, df, (x_left, y_left), e, tbl)
            krawczyk_system(f, df, (x_left, y_rigth), e, tbl)
            krawczyk_system(f, df, (x_right, y_left), e, tbl)
            krawczyk_system(f, df, (x_right, y_rigth), e, tbl)
        return

    f1m, f2m = f(x_middle, y_middle)
    df_mid = df(x_middle, y_middle)
    df1x,  df1y,  df2x,  df2y =  *df_xy[0], *df_xy[1]
    df1xm, df1ym, df2xm, df2ym = *df_mid[0], *df_mid[1]
    det = calculate_determinant(df_mid)
    retard_x = (1 + df2x*df1ym/det - df1x*df2ym/det)*(x-x_middle) \
                + ( df2y*df1ym/det - df1y*df2ym/det)*(y-y_middle)
    retard_y = (     - df2x*df1xm/det + df1x*df2xm/det)*(x-x_middle) \
                + (1 - df2y*df1xm/det + df1y*df2xm/det)*(y-y_middle)
    K_x = x_middle + ((-df2ym/det)*f1m + ( df1ym/det)*f2m ) + retard_x
    K_y = y_middle + (( df2xm/det)*f1m + (-df1xm/det)*f2m ) + retard_y
    x_next = K_x & x
    y_next = K_y & y
        
    # Stop if x_i+1 or y_i+1 is empty
    if not x_next or not y_next:
        to_table(x, x_width, y, y_width, '', tbl)
        return

    # Continue with narrowed interval
    to_table(x, x_width, y, y_width, '', tbl)
    krawczyk_system(f, df, (x_next, y_next), e, tbl)

In [100]:
func = lambda x, y: (x**2 - y - 3, x - y - 1)
dfunc = lambda x, y: ((2*x, -1), (1, -1))
init_intervals = (interval[-4, 3.5], interval[-3.7, 5.4])
e = 1e-6

In [101]:
krawczyk_system_result = pd.DataFrame(columns=['x', 'x width', 'y', 'y width', 'root'])
krawczyk_system(func, dfunc, init_intervals, e, krawczyk_system_result)
krawczyk_system_result

,x,x width,y,y width,root
0,"[-4.0000000, 3.5000000]",7.500000e+00,"[-3.7000000, 5.4000000]",9.100000e+00,
1,"[-4.0000000, -0.2500000]",3.750000e+00,"[-3.7000000, 0.8500000]",4.550000e+00,
2,"[-2.5803571, -0.2500000]",2.330357e+00,"[-3.5803571, -0.9017857]",2.678571e+00,
3,"[-1.7538867, -0.3361170]",1.417770e+00,"[-2.7538867, -1.3361170]",1.417770e+00,
4,"[-1.3259092, -0.6754016]",6.505076e-01,"[-2.3259092, -1.6754016]",6.505076e-01,
5,"[-1.0704960, -0.9295042]",1.409918e-01,"[-2.0704960, -1.9295042]",1.409918e-01,
6,"[-1.0033131, -0.9966869]",6.626227e-03,"[-2.0033131, -1.9966869]",6.626227e-03,
7,"[-1.0000073, -0.9999927]",1.463563e-05,"[-2.0000073, -1.9999927]",1.463563e-05,
8,"[-1.0000000, -1.0000000]",7.140066e-11,"[-2.0000000, -2.0000000]",7.140066e-11,Root
9,"[-4.0000000, -0.2500000]",3.750000e+00,"[0.8500000, 5.4000000]",4.550000e+00,
